<a href="https://www.kaggle.com/code/cihangirayoner/premier-league-analysis?scriptVersionId=137272491" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import warnings
import random
import base64
from collections import defaultdict
import matplotlib.pyplot as plt
import requests
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import math
from collections import defaultdict
import pprint
import re
import fnmatch
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
# Target of this work is to create as much as analytics from a semi-structured JSON format dataset
# Initial target is to create metrics and visualizing it


#Team Selection
#Team = str(input("Select team to see stats: ")).title().replace('And','and')
Team_Selection_Options = ['Arsenal', 'Everton', 'Leicester City', 'Manchester United', 'Brentford', 'Brighton and Hove Albion', 'Tottenham Hotspur', 
 'Nottingham Forest', 'West Ham United', 'Liverpool', 'Wolverhampton Wanderers', 'Bournemouth', 'Fulham', 'Manchester City', 'Newcastle United', 'Leeds United', 'Chelsea', 'Aston Villa', 'Crystal Palace', 'Southampton']
Team = random.choice(Team_Selection_Options).title().replace('And','and')
print("Selected Team:",Team)
Team_2_Names = Team.split(" ")


try:
    with open("../input/pl-league/epl_2022_2023_07_02_2023.json", 'r',encoding="UTF-8") as my_file:
        data = my_file.read().replace('&','and')
        obj = json.loads(data)
except UnicodeDecodeError:
    print("Content could not be read could be UTF-8 non compliance")
else:
    #Getting game ids
    Game_ids = []
    for key, value in obj.items(): 
        Game_ids.append(key)
    
    #Unique Team List
    Available_teams = []
    for key , value in obj.items():
        Available_teams.append(obj[key]["team1_name"])
    unique_teams = list(dict.fromkeys(Available_teams))
    
    def Total_Goals(Team): #Calculating Total Team Goals
        Team_Scores = [obj[str(i)]["event"][0] for i in Game_ids if (Team) in obj[str(i)]["event"][0]]#Getting Team Match Results
        Stripped = [item.split(" ") for item in Team_Scores]
        Goals = 0
        for i in range(len(Stripped)): 
            if len(Team_2_Names) > 1 or Team in Team_Scores:
                position = (Stripped[i].index((Team_2_Names[0])))
                Goals += int((Stripped[i][position + len(Team_2_Names)]).replace(",","").replace(".",""))
            else:
                position = (Stripped[i].index((Team)))
                Goals += int((Stripped[i][position + 1]).replace(",","").replace(".",""))
        return Goals
    

    def Total_Goals_2(function): ##Decorating on another function to find percentage of a team over total goals
        def Total_Goals_3(Team): #Calculating Total Team Goals
            Team_Scores = [obj[str(i)]["event"][0] for i in Game_ids if (Team) in obj[str(i)]["event"][0]]#Getting Team Match Results
            Stripped = [item.split(" ") for item in Team_Scores]
            Goals = 0
            for i in range(len(Stripped)): 
                if len(Team_2_Names) > 1 or Team in Team_Scores:
                    position = (Stripped[i].index((Team_2_Names[0])))
                    Goals += int((Stripped[i][position + len(Team_2_Names)]).replace(",","").replace(".",""))
                else:
                    position = (Stripped[i].index((Team)))
                    Goals += int((Stripped[i][position + 1]).replace(",","").replace(".",""))
            TotalGoals = function()
            if TotalGoals == 0:
                percent = 0
            else:
                percent = Goals/TotalGoals
                percentage = "{:.2%}".format(percent)
            print("Percentage of Team`s Goal over total: ",percentage)
            return function
        return Total_Goals_3
   
    def Total_Shots(Team): #Calculating Total Team Shoots
        Shoots = 0
        for i in Game_ids:
            if (Team) == obj[str(i)]["team2_name"]:
                Shoots += int((obj[str(i)]['team2_stat']['shots']))
            elif (Team) == obj[str(i)]["team1_name"]:
                Shoots = Shoots + int((obj[str(i)]["team1_stat"]["shots"]))
        return Shoots
        
    def Shot_Goal_Percentage(Total_Goals,Total_Shots,Team): #Calculating Successful Shots
        Total_Goals = int(Total_Goals(Team))
        Total_Shoots = int(Total_Shots(Team))
        if Total_Shoots == 0:
            percent = 0
        else:
            percent = Total_Goals/Total_Shoots
        percentage = "{:.2%}".format(percent)
        return percentage
    
    def Most_10_starting_players(a=obj): #List of 10 players among league has most starting lineup
        Players_startings = [obj[str(i)]["team1_startings"] for i in Game_ids]
        Players_startings.extend(obj[str(i)]["team2_startings"] for i in Game_ids)
        Player_startings_2 = []
        for i in Players_startings:
            for a in i:
                Player_startings_2.append(a)
        Unique_players = pd.Series(Player_startings_2).drop_duplicates()
        dicts_players = {}
        for i in Unique_players:
            dicts_players[i] = [Player_startings_2.count(i)]
        sorted_footballers_by_goals = sorted(dicts_players.items(), key=lambda x:x[1], reverse = True)[:10]
        df = pd.DataFrame(data=sorted_footballers_by_goals,columns=['Player_Name','Games_Started'])
        return df
    
    @Total_Goals_2 #Decorating on another function to find percentage of a team over total goals
    def Total_Goal_Score_Number(b=obj):
        Total_Goals_All_Teams = 0
        a = [obj[str(i)]["event"][0] for i in Game_ids]
        for i in a:
            goal_list = re.findall(r"[\w']+", i)
            for i in range(len(goal_list)):
                if goal_list[i].isdecimal():
                    Total_Goals_All_Teams += int(goal_list[i])
        #print(Total_Goals_All_Teams)
        return Total_Goals_All_Teams
    
    def Score_Distribution_by_Teams(b=obj): #Shows all goals made by teams
        dict_team_goals_2 = {}
        for a in unique_teams:
            Team_Scores = [obj[str(i)]["event"][0] for i in Game_ids if a in obj[str(i)]["event"][0]] #Getting Team Match Results
            Stripped = [item.split(" ") for item in Team_Scores]
            a = a.split(' ')
            Goals = 0
            for i in range(len(Stripped)): 
                if len(a) > 1 or a in Team_Scores:
                    position = (Stripped[i].index((a[0])))
                    Goals += int((Stripped[i][position + len(a)]).replace(",","").replace(".",""))
                    dict_team_goals_2[str(a)] = [Goals]
                elif len(a) == 1:
                #a = str(a).replace("[","").replace("]","")
                    position = (Stripped[i].index(a[0]))
                    Goals += int((Stripped[i][position + 1]).replace(",","").replace(".",""))
                    dict_team_goals_2[str(a)] = [Goals]
        sorted_teams_by_goals = sorted(dict_team_goals_2.items(), key=lambda x:x[1], reverse = True)
        df = pd.DataFrame(data=sorted_teams_by_goals,columns=['Team_Name','Goals'])
        return df

    



# print("Total Shots:",Total_Shots(Team))
print("Total Goals:",Total_Goals(Team))
print(Score_Distribution_by_Teams())
# print("Shot to Goal Percentage:",Shot_Goal_Percentage(Total_Goals,Total_Shots,Team))
# print(Most_10_starting_players())
Total_Goal_Score_Number(Team)

##########PANDAS EXERCISES################

#Normalized JSON data and created structured dataset
Normalized = pd.concat([pd.json_normalize(obj[str(i)]) for i in Game_ids])
Normalized

warnings.filterwarnings('ignore') # Hide Warnings

#Dimensions of the Dataset
Normalized.shape

#Cleaning non-numeric value holding rows to get clean dataset from columns 'team1_stat.possession_%' & 'team2_stat.possession_%' and creating a df
Normalized['team1_stat.possession_%'] = pd.to_numeric(Normalized['team1_stat.possession_%'], errors='coerce')
Normalized_2 = Normalized.dropna(subset=['team1_stat.possession_%'])
Normalized['team2_stat.possession_%'] = pd.to_numeric(Normalized['team2_stat.possession_%'], errors='coerce')
Normalized_2 = Normalized.dropna(subset=['team2_stat.possession_%'])

#Checking first 5 rows
Normalized_2.head()

#Converting non-numeric columns to numeric data types for analytical purposes
Normalized_2.loc[:,'team1_stat.possession_%':] = Normalized_2.loc[:,'team1_stat.possession_%':].apply(pd.to_numeric, errors = 'coerce')

#Checking datatypes
Normalized_2.dtypes


#Top 10 Most Shoot MadeTeam

Normalized_2["Total_Shoots"] = Normalized_2.loc[:,['team1_stat.shots','team2_stat.shots']].sum(axis=1)

Normalized_2 = Normalized_2.groupby(['team1_name']).sum()

Normalized_2['Shoot_rank'] = Normalized_2['Total_Shoots'].rank(ascending=False)

Normalized_2 = Normalized_2.sort_values(by='Shoot_rank', ascending=True)

Top_10_shooter_teams = Normalized_2[['team1_stat.shots','team2_stat.shots','Total_Shoots','Shoot_rank']]

Top_10_shooter_teams = Top_10_shooter_teams.head(10)

#print(Top_10_shooter_teams.head(1)) # TOP 1




/kaggle/input/pl-league/epl_2022_2023_07_02_2023.json
/kaggle/input/epl-dataset-20222023-update-every-week/player_info.csv
/kaggle/input/epl-dataset-20222023-update-every-week/epl_2022_2023_07_02_2023.json
Selected Team: Nottingham Forest
Total Goals: 17
                                Team_Name Goals
0                  ['Manchester', 'City']  [54]
1                             ['Arsenal']  [45]
2                ['Tottenham', 'Hotspur']  [41]
3   ['Brighton', 'and', 'Hove', 'Albion']  [38]
4                ['Manchester', 'United']  [37]
5                           ['Brentford']  [35]
6                           ['Liverpool']  [34]
7                 ['Newcastle', 'United']  [34]
8                   ['Leicester', 'City']  [32]
9                              ['Fulham']  [32]
10                    ['Leeds', 'United']  [26]
11                     ['Aston', 'Villa']  [25]
12                            ['Chelsea']  [22]
13                        ['Bournemouth']  [19]
14                  ['Cry